In [4]:
# from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI, ChatGooglePalm, ChatVertexAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, LLMChain
# from langchain.agents import AgentExecutor
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from tools import retrival_question_answering
import streamlit as st
from utilities import *
from prompts import *
import uuid
import json

set_API(name="chatgroq", platform="c")
set_API(name="huggingface", platform="hf")
set_API(name="serpapi", platform="s")

dict_ignore_files = {
    "dir": [".github", ".git", ".devcontainer", "docs", "examples", "sample_documents"],
    "extension": ["faiss", "pdf", "cff","png", "jpg", "gif", "yaml", "cff", "Dockerfile", "lock", "toml", "odt", "gitmodules", "csv", "example", "ambr", "html", "typed", "xml", "yml", "avi"],
    "file": [".gitignore", ".dockerignore", ".flake8", ".gitattributes", "Dockerfile", "LICENSE", "Makefile"]
}
# st.title("DirectoryGenius Chatbot Omega")
model2 = ChatGroq( model_name="Llama3-70b-8192")#Mixtral-8x7b-32768

Done
Done
Done


In [5]:
embedd_model = "all-mpnet-base-v2"
cache_folder_name = "cache/mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedd_model, cache_folder=cache_folder_name)

/home/roy/miniconda3/envs/PyTorch_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
folder_path = "/home/roy/GSOC/TV-News-Chat-LLM"
strut_file_path = dir_structure(path_to_dir = folder_path, ignore_files = {"dir": [], "extension": [], "file": []})
summary_file_path = summarize_and_write(path_to_dir=folder_path, ignore_files = dict_ignore_files, llm = "Llama3-70b-8192")
databasename = folder_path.split("/")[-1]

Output()

Output()

In [9]:
files = Search2(path = folder_path, ignore=dict_ignore_files)
Create_DB(files=files, database_path = f"Database/{databasename}-{cache_folder_name}", embeddings=embeddings)
os.remove(strut_file_path)
os.remove(summary_file_path )

Output()

Output()

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.
Creating 173 embeddings in 1 batches of size 173:: 100%|██████████| 1/1 [00:08<00:00,  8.02s/it]

Dataset(path='Database/TV-News-Chat-LLM-cache/mpnet-base-v2', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (173, 1)     str     None   
 metadata     json      (173, 1)     str     None   
 embedding  embedding  (173, 768)  float32   None   
    id        text      (173, 1)     str     None   


In [18]:
db2 = DeepLake(dataset_path=f"Database/{databasename}-{cache_folder_name}", read_only=True, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in Database/langchain-cache/mpnet-base-v2 already exists, loading from the storage


In [19]:
retriever = db2.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
# retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

In [20]:
def open_GUI(file: str):#, application: str = "xdg-open"):

    os.system(f"xdg-open {file}")

    
def find_similar_files(string: str):

    context = retriever.get_relevant_documents(string)
    found = []
    for cont in context:
        found.append(cont.page_content)
    if found:
        return "?".join(found)
    else:
        return ""
    
def find_exact_files(string: str):
    command = ["grep", "-r", string, "Database/langchain/"]
    for dir in dict_ignore_files["dir"]:
        command.extend([f"--exclude-dir={dir}"])
        # pass
    # print(command)
    result = subprocess.run(command, stdout=subprocess.PIPE).stdout.decode().split("\n")
    if result:
        files = map(lambda x: x.split(":")[0], result)
        return "?".join(files)
    else:
        return ""

In [11]:
# !pip install google-search-results

In [21]:
serp_search = SerpAPIWrapper()
tools = [
    Tool.from_function(
        func=open_GUI,
        name = "Open_file_in_editor",
        description="useful for when you need to open a file in editor"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=find_exact_files,
        name = "Search_Exact_Files",
        description="useful for when you need to search for a code snippet in a database. in case no file is found, a empty string will be returned in case of more than one file, their names will be separated by '?' symbol."
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=find_similar_files,
        name = "Search_Similar_Files",
        description="useful for when you need to search for a file related to some context and you don't have the exact keyword to search. in case of more than one file, their names will be separated by '?' symbol. in case no file is found, a empty string will be returned"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=serp_search.run,
        name = "Search ",
        description="useful when you need to search for a topic on internet. use it only when its absolute necessary."
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

In [22]:
# model2 = ChatGroq( model_name="Llama3-70b-8192")#Mixtral-8x7b-32768
memory = ConversationBufferMemory(
    llm=model2,
    memory_key="chat_history",
    return_messages=True,
    max_token_limit=2000
)
# memory.output_key = 'result'
# memory2 = ConversationSummaryBufferMemory(llm = model2,max_token_limit=200)
# memory2.output_key = 'result'

In [23]:
condense_title_chain = LLMChain(
            llm=model2,
            prompt=CONDENSE_title_PROMPT,
            # verbose=verbose,
            # callbacks=callbacks,
        )

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    model2,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

with open("chat_IDs.json", "w") as file:
        json.dump({"details": []}, file, indent = 4)


In [27]:
question = "show the tree structure of the langchain"
reply = qa_chain({"query": question} )
display_output(reply["result"])

Based on the provided file contents, here is the tree structure of the LangChain:
```
/home/roy/Langchain/langchain
├── .folder_structure.md
├── README.md
├── libs
│   └── langchain
│       ├── __init__.py
│       ├── chains
│       │   ├── base.py
│       │   └── llm.py
│       ├── langchain
│       │   ├── __init__.py
│       │   └── smith
│       │       └── evaluation.py
│       ├── langchain_core
│       │   ├── __init__.py
│       │   ├── language_models
│       │   │   └── llms
│       │   │       └── base.py
│       │   ├── load
│       │   │   ├── __init__.py
│       │   │   └── dump.py
│       │   ├── messages
│       │   │   └── base.py
│       │   ├── outputs
│       │   │   └── base.py
│       │   ├── prompts
│       │   │   └── prompt.py
│       │   ├── retrievers
│       │   │   └── base.py
│       │   ├── schema
│       │   │   └── base.py
│       │   ├── utils
│       │   │   ├── __init__.py
│       │   │   ├── env.py
│       │   │   ├── formatting.py
│       │   │   ├── input.py
│       │   │   ├── loading.py
│       │   │   └── strings.py
│       │   └── _api.py
│       └── cookbook
│           ├── causal_program_aided_language_model.ipynb
│           ├── langgraph_crag.ipynb
│           └── langgraph_self_rag.ipynb
```
Note that this structure might not be exhaustive, as not all files and directories might be present in the provided content.

If you have any further questions or need clarification on any specific part of the structure, feel free to ask! 

In [62]:

question = input("Ask query:")
content = {"Human": '',
           "Assistant": ''}
unique_id1 = uuid.uuid1()
with open(f"memory/{unique_id1}.json", "w") as file:
        json.dump({"history": []}, file, indent = 4)
reply = qa_chain({"query": question} )
text = condense_title_chain({"question": question, "reply" : reply["result"]})
with open("chat_IDs.json", 'r') as file:
        id = json.load(file)
id["details"].append({text["text"]: str(unique_id1)})
with open("chat_IDs.json", "w") as file:
        json.dump(id, file, indent = 4)
while question:
    result = qa_chain({"query": question} )
    with open(f"memory/{unique_id1}.json", 'r') as file:
        data = json.load(file)
    print(data)
    content["Human"] = question
    content["Assistant"] = result["result"]
    data["history"].append(content)
    display_output(result=result["result"])
    with open(f"memory/{unique_id1}.json", "w") as file:
        json.dump(data, file, indent = 4)
    question = input("Ask query:")
    

/home/roy/miniconda3/envs/PyTorch_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [171]:
result

{'query': 'Show me the whole tree structure of opencv directory make sure it include all the files',
 'result': 'Here is the tree structure of the OpenCV directory:\n```\n/home/roy/Documents/Opencv/\n.ipynb_checkpoints/\n6_contour_and_shape_detection-checkpoint.ipynb\n1_edges_and_gradients.ipynb\n2_convolutions_and_morphological.ipynb\n3_blurring_and_thresholding.ipynb\n4_drawing_and_bitwise.ipynb\n5_colour_detection.ipynb\n6_contour_and_shape_detection.ipynb\n7_mouse_events_and_wrap.ipynb\n8_haarcascades.ipynb\nutilities.py\nassets/\ncascades/\nhaarcascade_smile.xml\nhaarcascade_righteye_2splits.xml\nhaarcascade_profileface.xml\nhaarcascade_lowerbody.xml\nhaarcascade_eye.xml\nhaarcascade_frontalface_alt.xml\nhaarcascade_russian_plate_number.xml\nhaarcascade_fullbody.xml\nhaarcascade_lefteye_2splits.xml\nhaarcascade_eye_tree_eyeglasses.xml\nhaarcascade_frontalcatface.xml\nhaarcascade_upperbody.xml\nhaarcascade_frontalface_alt2.xml\nhaarcascade_frontalcatface_extended.xml\nhaarcascade_f

In [149]:
qa = ConversationalRetrievalChain.from_llm(
    model2,
    retriever=retriever,
    memory=memory,
    # verbose = True,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    # chain_type = "stuff"
)


In [145]:
qa.input_keys

['question', 'chat_history']

In [150]:
result = {"chat_history": ''}
question = "Hello myself Aniruddha"
text = condense_question_chain({"chat_history": result["chat_history"] , "question": question})

result = qa({"question": text["text"]})
# memory.save_context({"input": result["chat_history"][0::2]}, {"output": result["chat_history"][1::2]})
display_output(result=result['answer'])

Hello Aniruddha! It's nice to meet you. Is there something I can help you with or would you like to ask me a question? 

In [172]:
result

{'query': 'Show me the whole tree structure of opencv directory make sure it include all the files',
 'result': 'Here is the tree structure of the OpenCV directory:\n```\n/home/roy/Documents/Opencv/\n.ipynb_checkpoints/\n6_contour_and_shape_detection-checkpoint.ipynb\n1_edges_and_gradients.ipynb\n2_convolutions_and_morphological.ipynb\n3_blurring_and_thresholding.ipynb\n4_drawing_and_bitwise.ipynb\n5_colour_detection.ipynb\n6_contour_and_shape_detection.ipynb\n7_mouse_events_and_wrap.ipynb\n8_haarcascades.ipynb\nutilities.py\nassets/\ncascades/\nhaarcascade_smile.xml\nhaarcascade_righteye_2splits.xml\nhaarcascade_profileface.xml\nhaarcascade_lowerbody.xml\nhaarcascade_eye.xml\nhaarcascade_frontalface_alt.xml\nhaarcascade_russian_plate_number.xml\nhaarcascade_fullbody.xml\nhaarcascade_lefteye_2splits.xml\nhaarcascade_eye_tree_eyeglasses.xml\nhaarcascade_frontalcatface.xml\nhaarcascade_upperbody.xml\nhaarcascade_frontalface_alt2.xml\nhaarcascade_frontalcatface_extended.xml\nhaarcascade_f

In [173]:
question = "Show me the whole tree structure of opencv directory make sure it include all the files even from subdirectories"
result = qa({"question": question})
# memory.save_context({"input": result["chat_history"][0::2]}, {"output": result["chat_history"][1::2]})
display_output(result=result['answer'])

Hello Aniruddha!

Based on the provided content, here is the tree structure of the OpenCV directory:

```
/home/roy/Documents/Opencv/
├── .folder_structure.md
├── 1_edges_and_gradients.ipynb
├── 2_convolutions_and_morphological.ipynb
├── 3_blurring_and_thresholding.ipynb
├── 4_drawing_and_bitwise.ipynb
├── 5_colour_detection.ipynb
├── 6_contour_and_shape_detection.ipynb
├── 7_mouse_events_and_wrap.ipynb
├── 8_haarcascades.ipynb
├── assets/
│   ├── cascades/
│   │   ├── haarcascade_eye.xml
│   │   ├── haarcascade_eye_tree_eyeglasses.xml
│   │   ├── haarcascade_frontalcatface.xml
│   │   ├── haarcascade_frontalface_alt.xml
│   │   ├── haarcascade_fullbody.xml
│   │   ├── haarcascade_lefteye_2splits.xml
│   │   ├── haarcascade_lowerbody.xml
│   │   ├── haarcascade_profileface.xml
│   │   ├── haarcascade_righteye_2splits.xml
│   │   └── haarcascade_russian_plate_number.xml
│   ├── doc.py
│   ├── images/
│   │   ├── cards.jpg
│   │   ├── faces.jpg
│   │   ├── rect.jpg
│   │   └── shapes.jpg
│   └── videos/
│       ├── cars.avi
│       ├── output.avi
│       └── walking.avi
├── utilities.py
└── .ipynb_checkpoints/
    ├── 6_contour_and_shape_detection-checkpoint.ipynb
    └── ... (other checkpoint files)
```

Note: The `.ipynb_checkpoints` directory may contain more files, but they are not explicitly listed in the provided content. 

In [176]:
type(result["chat_history"][0])

langchain_core.messages.human.HumanMessage

In [142]:
question = "Now did you remember my name?"
result = qa({"question": question})
display_output(result=result['answer'])

Hello Aniruddha!

I'd be happy to help you with the tree structure of the OpenCV directory. Based on the provided file content, here is the tree structure:

```
/home/roy/Documents/Opencv/
|- .folder_structure.md
|- assets/
|  |- doc.py
|  |- images/
|  |  |- shapes.jpg
|  |  |- hello_noise_white.jpg
|  |  |- examples kernels.png
|  |- cascades/
|  |  |- haarcascade_smile.xml
|  |  |- haarcascade_righteye_2splits.xml
|  |  |- haarcascade_profileface.xml
|  |  |- haarcascade_lowerbody.xml
|  |  |- haarcascade_eye.xml
|  |  |- haarcascade_frontalface_alt.xml
|  |  |- haarcascade_russian_plate_number.xml
|  |  |- haarcascade_fullbody.xml
|  |  |- haarcascade_lefteye_2splits.xml
|  |  |- haarcascade_eye_tree_eyeglasses.xml
|  |  |- haarcascade_frontalcatface.xml
|- 2_convolutions_and_morphological.ipynb
|- 6_contour_and_shape_detection-checkpoint.ipynb
|- 8_haarcascades.ipynb
|- utilities.py
```

I've included all the files and subdirectories mentioned in the provided content. Let me know if you have any further questions!

And, yes, I remember your name: Aniruddha! 

In [23]:
question = "Again doing same thing why don't you show me whole structure?"
result = qa({"question": question})
display_output(result=result['answer'])

Based on the provided file contents, here is the tree structure of the OpenCV directory:

```
/home/roy/Documents/Opencv/
assets/
doc.py
folder_structure.md
utilities.py
2_convolutions_and_morphological.ipynb
6_contour_and_shape_detection-checkpoint.ipynb
.ipynb_checkpoints/
6_contour_and_shape_detection-checkpoint.ipynb
```

Please note that this structure only includes the files and directories mentioned in the provided file contents. If there are other files or directories in the OpenCV directory, they are not included in this structure. 